In [ ]:
# 5.必须使用python编程，采用sklearn库，如果采用以前的spark 机器学习或者alink 编写得0分。
#建立前列腺癌Logistic回归预测模型,评估患者患前列腺癌的可能性,提高可疑前列腺癌患者的临床诊断率,为可疑前列腺癌患者进行前列腺穿刺提供更充分的依据。方法：回顾性分析2007年3月-2010年3月在我院行经直肠超声引导下前列腺穿刺活检的符合本研究标准的105例资料,其中前列腺癌为42例,非癌患者63例。按照穿刺时间先后顺序分组,前2/3病例(70例)分入建模组,后1/3(35例)病例分入验证组。分析建模组病例年龄、前列腺特异性抗原(PSA)、游离PSA比(f/t PSA)、PSA密度(PSAD)、肛门指检(DRE)及经直肠超声(TRUS)等指标对诊断前列腺癌的价值,综合以上指标,利用SPSS软件建立Logistic回归模型。利用验证组病例数据建立Logistic回归模型预测概率的受试者工作特征(ROC)曲线,分析其相应的敏感度、特异度,确定临界值。结果：本前列腺癌Logistic回归预测模型以穿刺病理是否为前列腺癌（lpsa数据 第一列代表目标值 其他列都是特征）

In [7]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
# 1.加载前列腺癌数据，构建会话对象,lpsa读取样本数据，并且将lable连续型变量转化为0,1 [0代表正常，1代表异常](5分)
import  pandas as pd
import  numpy as np
lpsa=pd.read_csv("E:///python代码/lx-yk/data-yk/yk2/lpsa.data",names=['1','2','3','4','5','6','7','8','9'])
#并且将lable连续型变量转化为0,1 
# lpsa数据 第一列代表目标值 其他列都是特征
from sklearn.preprocessing import Binarizer
bina=Binarizer(threshold=2)
zh=bina.fit_transform(lpsa.iloc[:,0].values.reshape(-1,1))
lpsa['lable']=zh
lpsa
# 2训练时需要LabeledPoint这样格式的数据，自主划分训练集、测试集(5分)
x=lpsa.drop(['lable','1'],axis=1)# X = lpsa.iloc[:,1:-1]
y=lpsa['lable']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# 3.自主选择两种算法，构造训练模型(8分)
# [0代表正常，1代表异常]所以使用分类
#-----------------------------------------------决策树----------------------------------------------------
from sklearn.tree import DecisionTreeClassifier
trees=DecisionTreeClassifier(min_samples_split=3,min_samples_leaf=2)
trees.fit(X_train,y_train)


#-----------------------------------------------随机森林---------------------------------------------------
from sklearn.ensemble import RandomForestClassifier
tree=RandomForestClassifier(min_samples_split=3,min_samples_leaf=2)
tree.fit(X_train,y_train)

# 4、对以上两种算法进行参数调优（8分）
#--------------------------决策树------------------------调优-------------------------------------------------
trees_ty={'criterion':['gini','entropy'],'splitter':['best','random'],'min_samples_split':[2,3],'max_depth':[30,50]}
clf1 = GridSearchCV(estimator=trees,param_grid=trees_ty,cv=5)
clf1.fit(X_train,y_train)# 训练模型
# 获取最优模型
trees_zy=clf1.best_estimator_
print("决策树调优获取的最优模型",trees_ty)
#-------------------------随机森林-----------------------调优--------------------------------------------------
tree_ty={'n_estimators':[100,200],'min_samples_split':[2,3],'max_depth':[30,50]}
clf2 = GridSearchCV(estimator=tree,param_grid=tree_ty,cv=5)
clf2.fit(X_train,y_train)# 训练模型
# 获取最优模型
tree_zy=clf2.best_estimator_
print("随机森林调优获取的最优模型",tree_zy)
# 5、对以上两种模型评估泛化能力，至少使用两种评估指标（8分）
#--------------------------决策树------------------------调优-------------------------------------------------
y_pred_trees = trees_zy.predict(X_test)
from sklearn.metrics import accuracy_score
jq1=accuracy_score(y_test, y_pred_trees)
from sklearn.metrics import precision_score
zq1=precision_score(y_test, y_pred_trees)
print('决策树的准确率',jq1)
print('决策树的精确率',jq1)
#-------------------------随机森林-----------------------调优--------------------------------------------------
y_pred_tree=tree_zy.predict(X_test)

jq2=accuracy_score(y_test, y_pred_tree)
zq2=precision_score(y_test, y_pred_tree)
print('随机森林的准确率',jq1)
print('随机森林的精确率',jq1)
from sklearn.metrics import accuracy_score
# 6、输出以上两种模型最优参数（5分）
trees_zys=clf1.best_estimator_
print("决策树最优参数",trees_zys)
tree_zys=clf2.best_estimator_
print("随机森林最优参数",trees_zys)
print ("7、使用评估指标较优者，对样本批量预测（5分）")
t7_1=trees_zys.predict(X_test)
print(t7_1)
t7_2=tree_zys.predict(X_test)
print(t7_2)
print( "8  使用classification_report输出分类报告结果（4分）")
t8=classification_report(y_test,t7_1)
print(t8)





决策树调优获取的最优模型 {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_split': [2, 3], 'max_depth': [30, 50]}
随机森林调优获取的最优模型 RandomForestClassifier(max_depth=50, min_samples_leaf=2, min_samples_split=3,
                       n_estimators=200)
决策树的准确率 0.7142857142857143
决策树的精确率 0.7142857142857143
随机森林的准确率 0.7142857142857143
随机森林的精确率 0.7142857142857143
决策树最优参数 DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_leaf=2,
                       min_samples_split=3, splitter='random')
随机森林最优参数 DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_leaf=2,
                       min_samples_split=3, splitter='random')
7、使用评估指标较优者，对样本批量预测（5分）
[1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0.]
8  使用classification_report输出分类报告结果（4分）
              precision    recall  f1-score   support

         0.0       0.67      0.67      0.67         6
         1.0       0.75      0.75      0.75         8

    accuracy                           0.71        14
 

       0   2  3  1424380312
0      0   3  1  1424380312
1      0   5  2  1424380312
2      0   9  4  1424380312
3      0  11  1  1424380312
4      0  12  2  1424380312
...   ..  .. ..         ...
1495  29  90  4  1424380312
1496  29  93  1  1424380312
1497  29  94  4  1424380312
1498  29  97  1  1424380312
1499  29  99  1  1424380312

[1500 rows x 4 columns]
